In [1]:
# -*- coding: utf-8 -*-
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
import time
import itertools
import models
import tensorflow as tf
import numpy as np
from config import *
from utils_data import *
from utils import *
from sklearn.model_selection import train_test_split


train_data_path='bala_train'
dev_data_path='bala_dev'
test_data_path='bala_test'
bigram_words_path='bala_train_bigram'
config=DictConfig
DATA_PATH='data'
config.hidden_dim = 64



In [2]:
tf.flags.DEFINE_string('dataset','bala',"Dataset for evaluation")
tf.flags.DEFINE_string("model_path", 'bala_model_path', "The filename of model path")
tf.flags.DEFINE_float("memory",1.0,"Allowing GPU memory growth")
tf.flags.DEFINE_bool('is_train',True,"Train or predict")
tf.flags.DEFINE_integer('min_bg_freq',0,'The mininum bigram_words frequency')
tf.flags.DEFINE_string('general','bala_dict_1',"General dictionary.")
tf.flags.DEFINE_string('domain',None,"domain-specific dictionary")
tf.flags.DEFINE_string('model','DictHyperModel','Choose the model.')
FLAGS = tf.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [3]:
#对训练集的各个字和生成的bigram 赋予id  
#例子：word2id={'P': 1, 'E': 0, 'U': 3, 'S': 2,'李': 5,'鹏': 6, '在': 7, '北': 8,'京'：4}
word2id = get_word2id(train_data_path,bigram_words=bigram_words_path,min_bw_frequence=FLAGS.min_bg_freq)

#最终的X_train是n(训练集的字数)个9维的列表组成的list。
#X_train 9维数据中包含：以某个字为中心，窗口大小为5的词,包含的五个字的id，以及这个size=5窗口得到的4个bigram的id     
#y_train是各个字对应的分词标记BIES：TAGB,TAGI,TAGE,TAGS=0,1,2,3
X_train,y_train=get_train_data(train_data_path,word2id)
X_valid,y_valid=get_train_data(dev_data_path,word2id)
x_test, y_test = get_train_data(test_data_path, word2id)

#得到训练集每个字所对应的特征向量：
#代表以第i字为结尾的五字四字三字二字的词                  / 以第i字为开头的二字三字四字五字的词 
dict_train=generate_dicttag(train_data_path,general_words_path=FLAGS.general,domain_words_path=FLAGS.domain)
dict_valid=generate_dicttag(dev_data_path,general_words_path=FLAGS.general,domain_words_path=FLAGS.domain)
dict_test=generate_dicttag(test_data_path,general_words_path=FLAGS.general,domain_words_path=FLAGS.domain)

#词向量矩阵 len(word2id * size)
init_embedding = get_embedding(word2id,size=config.word_dim)

oov character rate 0.002372


In [4]:
print( 'train_data_path: %s' % train_data_path )
print( 'valid_data_path: %s' % dev_data_path)
print( 'test_data_path: %s' % test_data_path)
print( 'bigram_words_path: %s' % bigram_words_path)
print( 'model_path: %s' % FLAGS.model_path)
print( 'min_bg_freq: %d'% FLAGS.min_bg_freq)

print( 'len(train_data): %d' % len(X_train))
print( 'len(valid_data): %d' % len(X_valid))
print( 'len(test_data): %d' % len(x_test))
print( 'init_embedding shape: [%d,%d]' % (init_embedding.shape[0], init_embedding.shape[1]))
print( 'Train started!')

train_data_path: bala_train
valid_data_path: bala_dev
test_data_path: bala_test
bigram_words_path: bala_train_bigram
model_path: bala_model_path
min_bg_freq: 0
len(train_data): 15
len(valid_data): 2
len(test_data): 17
init_embedding shape: [1686,100]
Train started!


In [5]:
tfConfig = tf.ConfigProto()
tfConfig.gpu_options.per_process_gpu_memory_fraction = FLAGS.memory

In [6]:
tf.reset_default_graph()
with tf.Session(config=tfConfig) as sess:
    model=getattr(models,FLAGS.model)(vocab_size=len(word2id),word_dim=config.word_dim,hidden_dim=config.hidden_dim,
                pad_word=word2id[PAD],init_embedding=init_embedding,num_classes=config.num_classes,clip=config.clip,
                lr=config.lr,l2_reg_lamda=config.l2_reg_lamda,num_layers=config.num_layers,rnn_cell=config.rnn_cell,
                bi_direction=config.bi_direction,
                                      hidden_dim2=config.hidden_dim2,
                                      hyper_embedding_size=config.hyper_embed_size)
    

    

C:\Users\Hello_World\AppData\Local\conda\conda\envs\py35\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [7]:
x_batch=[]
x_dict=[]
y_batch=[]
pad_word=word2id[PAD]
dropout_keep_prob=config.dropout_keep_prob
for step,(X,dict_X,Y) in enumerate(data_iterator2(zip(X_train,dict_train),y_train,2,padding_word=word2id[PAD],shuffle=True)):
    #print(step)
    x_batch=X
    x_dict=dict_X
    y_batch=Y
    
    
    #训练得到loss
    #loss=model.train_step(sess,X,dict_X,Y,config.dropout_keep_prob)
    #print( 'epoch:%d>>%2.2f%%' % (epoch,config.batch_size*step*100.0/len(X_train)),'completed in %.2f (sec) <<\r' % (time.time()-start_time),)
    #sys.stdout.flush()
    #train_loss.append(loss)
#train_loss=np.mean(train_loss,dtype=float)
#print( 'Train Epoch %d loss %f' % (epoch, train_loss))
print(pad_word)

316


In [8]:

np.set_printoptions(threshold=np.inf)
f=open('temp_print.txt','w')
print(x_batch,file=f)

In [9]:

x=tf.placeholder(dtype=tf.int32,shape=[None,None,9],name='input_x')   #input_x占位
y=tf.placeholder(dtype=tf.int32,shape=[None,None],name='input_y')   	#input_y占位
dict=tf.placeholder(dtype=tf.float32,shape=[None,None,8],name='dict')	#dict占位
dropout_keep_prob=tf.placeholder(dtype=tf.float32,name='dropout_keep_prob')  #dropout_keep_prob占位
seq_length=tf.reduce_sum(tf.cast(tf.not_equal(x[:,:,2], tf.ones_like(x[:,:,2])*pad_word), tf.int32), 1)
weights=tf.cast(tf.not_equal(x[:,:,2], tf.ones_like(x[:,:,2])*pad_word), tf.float32)
batch_size = tf.shape(x)[0]



In [10]:
feed_dict={ 
    x:x_batch,
    dict:x_dict,
    y:y_batch,
    dropout_keep_prob:0
}


In [11]:

x=tf.nn.embedding_lookup(init_embedding,x)
print(x.shape)
x = tf.reshape(x, [batch_size, -1, 9 * 100])
print(x.shape)


(?, ?, 9, 100)
(?, ?, 900)


In [17]:
print(x.shape)
print(dict.shape)
x = tf.cast(x,tf.float32)
inputx=tf.concat([x,dict],axis=2)    

print(inputx.shape)


(?, ?, 900)
(?, ?, 8)
(?, ?, 908)


In [ ]:
with tf.Session(config=tfConfig) as sess:
    f=open("temp_file2.txt",'w')
    
    print(sess.run(x,feed_dict),file=f)

sess.close()